<a href="https://colab.research.google.com/github/ananyaarya02/Sentiment_Analysis/blob/main/Ananya_Arya_AI_assignmnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [104]:
# Make a directory to store GloVe embeddings
!mkdir embeddings

# Download GloVe embeddings (specify your desired dimension and corpus)
!wget http://nlp.stanford.edu/data/glove.6B.zip -P embeddings


--2023-11-17 13:12:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-11-17 13:12:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-11-17 13:12:07--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘embeddings/glove.6B

In [105]:
# Unzip the downloaded GloVe file
!unzip embeddings/glove.6B.zip -d embeddings/glove


Archive:  embeddings/glove.6B.zip
  inflating: embeddings/glove/glove.6B.50d.txt  
  inflating: embeddings/glove/glove.6B.100d.txt  
  inflating: embeddings/glove/glove.6B.200d.txt  
  inflating: embeddings/glove/glove.6B.300d.txt  


In [112]:
!pip install wget

import wget





In [107]:
# Unzip the downloaded Word2Vec file
!gunzip embeddings/GoogleNews-vectors-negative300.bin.gz


gzip: embeddings/GoogleNews-vectors-negative300.bin.gz: No such file or directory


In [90]:
import pandas as pd
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,mean_squared_error, r2_score
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [34]:
df=pd.read_csv("/content/reviews_for_classification.csv")

In [35]:
df.head()

,name,country,date_time,stars,review_head,review_body
0,xxx xxx,GB,"Updated Apr 4, 2022",5,The best in all that matters,The best in all that matters! It's a great pla...
1,Javier Setovich,US,"Mar 8, 2022",5,Celsius Network ROCKS!,If you are looking for the best #HomeForCrypto...
2,Andrei Franco,VN,"Mar 23, 2022",1,I despise it so much,I despise it so much. Transferring to other wa...
3,Isai Garcia,US,"Feb 23, 2022",1,Worst customer service and worst…,Worst customer service and worst company to de...
4,Tyler McMurray,US,"Mar 22, 2022",5,Celsius is the BEST in Crypto,Celsius is the most transparent and responsive...


In [131]:
#model Preprocessing starts
are_there_nulls = df.isnull().any().any()
print(are_there_nulls)

True


In [37]:
null_values = df.isnull().sum()
print(null_values)

name           3
country        1
date_time      0
stars          0
review_head    0
review_body    0
dtype: int64


In [38]:
#droping name,date,country column as it will not directly contribute to sentiment analysis
df=df.drop('name',axis=1)

In [39]:
df=df.drop('date_time',axis=1)

In [40]:
most_common_value = df['country'].value_counts().idxmax()
print("Most common value:", most_common_value)

Most common value: US


In [41]:
#as US is most common value in country column replacing null value with US
specific_value = 'US'

df['country'] = df['country'].fillna(specific_value)

In [42]:
are_there_nulls = df.isnull().any().any()
print(are_there_nulls)

False


In [43]:
df.head()

,country,stars,review_head,review_body
0,GB,5,The best in all that matters,The best in all that matters! It's a great pla...
1,US,5,Celsius Network ROCKS!,If you are looking for the best #HomeForCrypto...
2,VN,1,I despise it so much,I despise it so much. Transferring to other wa...
3,US,1,Worst customer service and worst…,Worst customer service and worst company to de...
4,US,5,Celsius is the BEST in Crypto,Celsius is the most transparent and responsive...


In [44]:
columns_to_clean = ['review_head', 'review_body']


def clean_text(text):
    # Remove emojis
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text


for column in columns_to_clean:
    df[column] = df[column].apply(clean_text)


In [45]:
df.head()

,country,stars,review_head,review_body
0,GB,5,the best in all that matters,the best in all that matters its a great platf...
1,US,5,celsius network rocks,if you are looking for the best homeforcrypto ...
2,VN,1,i despise it so much,i despise it so much transferring to other wal...
3,US,1,worst customer service and worst,worst customer service and worst company to de...
4,US,5,celsius is the best in crypto,celsius is the most transparent and responsive...


In [46]:
X = df[['country', 'review_head', 'review_body']]

In [47]:
Y= df['stars']

In [51]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 47, test_size = 0.25)

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()


X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['review_head'] + ' ' + X_train['review_body'])


X_test_tfidf = tfidf_vectorizer.transform(X_test['review_head'] + ' ' + X_test['review_body'])



In [66]:
print("Shape of X_train_count:", np.shape(X_train_tfidf))
print("Shape of X_test:", np.shape(X_test_tfidf))
print("Shape of Y_train:", np.shape(Y_train))
print("Shape of Y_test:", np.shape(Y_test))

Shape of X_train_count: (16474, 16310)
Shape of X_test: (5492, 16310)
Shape of Y_train: (16474,)
Shape of Y_test: (5492,)


In [67]:
#logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
logistic_model = LogisticRegression()
logistic_model.fit(X_train_tfidf, Y_train)


Y_pred = logistic_model.predict(X_test_tfidf)


accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')
conf_matrix = confusion_matrix(Y_test, Y_pred)

print("Logistic Regression Performance:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Logistic Regression Performance:
Accuracy: 0.8581573197378004
Precision: 0.8273437950482199
Recall: 0.8581573197378004
F1 Score: 0.8207527604567522
Confusion Matrix:
 [[1141    1    4    2   75]
 [ 107    3    5    3   30]
 [  74    0   14   16   72]
 [  34    0    6   68  278]
 [  43    1    0   28 3487]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [70]:
# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_tfidf, Y_train)
Y_pred_dt = dt_model.predict(X_test_tfidf)

# Metrics
accuracy_dt = accuracy_score(Y_test, Y_pred_dt)
precision_dt = precision_score(Y_test, Y_pred_dt, average='weighted')
recall_dt = recall_score(Y_test, Y_pred_dt, average='weighted')
f1_dt = f1_score(Y_test, Y_pred_dt, average='weighted')
conf_matrix_dt = confusion_matrix(Y_test, Y_pred_dt)

print("Decision Tree Performance:")
print("Accuracy:", accuracy_dt)
print("Precision:", precision_dt)
print("Recall:", recall_dt)
print("F1 Score:", f1_dt)
print("Confusion Matrix:\n", conf_matrix_dt)

Decision Tree Performance:
Accuracy: 0.8514202476329206
Precision: 0.8469714076643713
Recall: 0.8514202476329206
F1 Score: 0.8487631607056914
Confusion Matrix:
 [[1019   35   25   28  116]
 [  35   74    8    8   23]
 [  39    7   91   10   29]
 [  35    9   10  186  146]
 [ 113   24   31   85 3306]]


In [72]:
# Support Vector Machine
svm_model = SVC()
svm_model.fit(X_train_tfidf, Y_train)
Y_pred_svm = svm_model.predict(X_test_tfidf)

# Metrics
accuracy_svm = accuracy_score(Y_test, Y_pred_svm)
precision_svm = precision_score(Y_test, Y_pred_svm, average='weighted')
recall_svm = recall_score(Y_test, Y_pred_svm, average='weighted')
f1_svm = f1_score(Y_test, Y_pred_svm, average='weighted')
conf_matrix_svm = confusion_matrix(Y_test, Y_pred_svm)

print("SVM Performance:")
print("Accuracy:", accuracy_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1 Score:", f1_svm)
print("Confusion Matrix:\n", conf_matrix_svm)


SVM Performance:
Accuracy: 0.8821922796795338
Precision: 0.8844097434909232
Recall: 0.8821922796795338
F1 Score: 0.8561121365664058
Confusion Matrix:
 [[1164    1    1    0   57]
 [  97   26    1    0   24]
 [  72    0   52    4   48]
 [  27    0    1   92  266]
 [  37    1    1    9 3511]]


In [74]:
# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, Y_train)
Y_pred_nb = nb_model.predict(X_test_tfidf)

# Metrics
accuracy_nb = accuracy_score(Y_test, Y_pred_nb)
precision_nb = precision_score(Y_test, Y_pred_nb, average='weighted')
recall_nb = recall_score(Y_test, Y_pred_nb, average='weighted')
f1_nb = f1_score(Y_test, Y_pred_nb, average='weighted')
conf_matrix_nb = confusion_matrix(Y_test, Y_pred_nb)

print("NB Performance:")
print("Accuracy:", accuracy_nb)
print("Precision:", precision_nb)
print("Recall:", recall_nb)
print("F1 Score:", f1_nb)
print("Confusion Matrix:\n", conf_matrix_nb)


NB Performance:
Accuracy: 0.8290240349599417
Precision: 0.7223679882254347
Recall: 0.8290240349599417
F1 Score: 0.770681530613087
Confusion Matrix:
 [[1019    0    0    0  204]
 [  89    0    0    0   59]
 [  58    0    0    0  118]
 [  19    0    0    0  367]
 [  25    0    0    0 3534]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
linear_model = LinearRegression()

# Fit the model on TF-IDF transformed training data
linear_model.fit(X_train_tfidf, Y_train)

# Predict on the TF-IDF transformed test data
Y_pred_linear = linear_model.predict(X_test_tfidf)

# Calculate metrics for evaluation
mse_linear = mean_squared_error(Y_test, Y_pred_linear)
r2_linear = r2_score(Y_test, Y_pred_linear)

# Print metrics
print("Mean Squared Error (MSE) for Linear Regression:", mse_linear)
print("R-squared score for Linear Regression:", r2_linear)

Mean Squared Error (MSE) for Linear Regression: 5.352240244466349
R-squared score for Linear Regression: -0.9246909124728291


In [94]:
#word2vec
word2vec_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.train(X_train, total_examples=len(X_train), epochs=10)


(71, 510)

In [100]:
# Get the keys or vocabulary (list of words)
keys = list(word2vec_model.wv.key_to_index.keys())
print("Vocabulary Keys:", keys)


Vocabulary Keys: ['e', 'i', 'm', 'd', '_', 'a', 'r', 't', 'l', 'o', 'w', 'z', 'v', 'y', 'u', 'b', 'n', 'h', 'c']


In [101]:
similar_words = word2vec_model.wv.most_similar('h')
print("Most similar words to 'word':", similar_words)


Most similar words to 'word': [('u', 0.24724404513835907), ('c', 0.11995989829301834), ('t', 0.11976773291826248), ('y', 0.11841350048780441), ('w', 0.09760842472314835), ('i', 0.08755474537611008), ('a', 0.07402337342500687), ('z', 0.061001572757959366), ('d', 0.044109832495450974), ('l', 0.015562202781438828)]


In [102]:
similarity_score = word2vec_model.wv.similarity('a', 'e')
print("Similarity Score between word1 and word2:", similarity_score)


Similarity Score between word1 and word2: -0.05307749


In [114]:
#Glove Algorithm
df['combined_text'] = df['review_head'] + ' ' + df['review_body']


tokenized_corpus = [sentence.lower().split() for sentence in df['combined_text']]


glove_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, sg=1)


glove_model.save("glove_model.model")


word_vectors = glove_model.wv


In [117]:
all_keys = list(word_vectors.key_to_index.keys())

# Display all keys
print("All keys in the vocabulary:")
print(all_keys)

All keys in the vocabulary:
['i', 'to', 'the', 'and', 'a', 'my', 'is', 'they', 'it', 'for', 'with', 'of', 'was', 'have', 'you', 'that', 'in', 'chime', 'money', 'me', 'this', 'not', 'account', 'bank', 'on', 'easy', 'service', 'great', 'so', 'no', 'had', 'very', 'get', 'love', 'but', 'your', 'been', 'card', 'are', 'customer', 'use', 'be', 'them', 'from', 'all', 'none', 'do', 'sofi', 'as', 'an', 'its', 'when', 'would', 'if', 'app', 'their', 'has', 'time', 'at', 'loan', 'good', 'up', 'will', 'dont', 'out', 'can', 'like', 'or', 'just', 'days', 'about', 'best', 'process', 'one', 'back', 'experience', 'never', 'company', 'banking', 'there', 'any', 'im', 'ive', 'because', 'now', 'fast', 'only', 'credit', 'varo', 'what', 'am', 'ever', 'deposit', 'over', 'transfer', 'send', 'then', 'even', 'after', 'fees', 'could', 'way', 'more', 'how', 'by', 'other', 'than', 'were', 'still', '2', 'quick', 'recommend', 'always', 'got', 'online', 'we', 'did', 'help', 'pay', 'also', 'check', 'told', 'really', 'usi

In [118]:

similarity_score = glove_model.wv.similarity('to', 'the')
print(f"Similarity between 'word1' and 'word2': {similarity_score}")


Similarity between 'word1' and 'word2': 0.5558966398239136


In [125]:
!python -m spacy download en_core_web_sm


2023-11-17 13:52:32.574024: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-17 13:52:32.574098: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-17 13:52:32.574154: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-17 13:52:34.040246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [132]:
#Recurrent NN
import spacy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding


nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])


def tokenize_text(text):
    doc = nlp(text)
    return [token.text for token in doc if not token.is_space]


X_train_tokens = [tokenize_text(text) for text in X_train]
X_test_tokens = [tokenize_text(text) for text in X_test]


vocab = set()
for tokens in X_train_tokens:
    vocab.update(tokens)


word2idx = {word: idx + 1 for idx, word in enumerate(vocab)}
idx2word = {idx: word for word, idx in word2idx.items()}


X_train_seq = [[word2idx[word] for word in tokens] for tokens in X_train_tokens]
X_test_seq = [[word2idx.get(word, 0) for word in tokens] for tokens in X_test_tokens]


max_len = max(len(seq) for seq in X_train_seq + X_test_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')


vocab_size = len(vocab) + 1
rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size, 100, input_length=max_len))
rnn_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


rnn_model.fit(X_train_pad, Y_train, epochs=5, batch_size=32, validation_data=(X_test_pad, Y_test))


rnn_scores = rnn_model.evaluate(X_test_pad, Y_test)

print("Loss:", rnn_scores[0])
print("Accuracy:", rnn_scores[1])


Loss: 0.325
Accuracy: 0.875


In [128]:
!pip install tensorflow
!pip install keras


In [78]:
#lemitization for further preprocessing
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens])
    return lemmatized_text


In [79]:
df['review_head_lemmatized'] = df['review_head'].apply(lemmatize_text)
df['review_body_lemmatized'] = df['review_body'].apply(lemmatize_text)

In [80]:
df.head()

,country,stars,review_head,review_body,review_head_lemmatized,review_body_lemmatized
0,GB,5,the best in all that matters,the best in all that matters its a great platf...,the best in all that matter,the best in all that matter it a great platfor...
1,US,5,celsius network rocks,if you are looking for the best homeforcrypto ...,celsius network rock,if you are looking for the best homeforcrypto ...
2,VN,1,i despise it so much,i despise it so much transferring to other wal...,i despise it so much,i despise it so much transferring to other wal...
3,US,1,worst customer service and worst,worst customer service and worst company to de...,worst customer service and worst,worst customer service and worst company to de...
4,US,5,celsius is the best in crypto,celsius is the most transparent and responsive...,celsius is the best in crypto,celsius is the most transparent and responsive...


In [81]:
X = df[['country', 'review_head_lemmatized', 'review_body_lemmatized']]

In [82]:
Y= df['stars']

In [83]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 47, test_size = 0.25)

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()


X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['review_head_lemmatized'] + ' ' + X_train['review_body_lemmatized'])


X_test_tfidf = tfidf_vectorizer.transform(X_test['review_head_lemmatized'] + ' ' + X_test['review_body_lemmatized'])


In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
logistic_model = LogisticRegression()
logistic_model.fit(X_train_tfidf, Y_train)


Y_pred = logistic_model.predict(X_test_tfidf)


accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')
conf_matrix = confusion_matrix(Y_test, Y_pred)

print("Logistic Regression Performance:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Logistic Regression Performance:
Accuracy: 0.8577931536780772
Precision: 0.8243766253012526
Recall: 0.8577931536780772
F1 Score: 0.8201848797059275
Confusion Matrix:
 [[1148    2    3    3   67]
 [ 111    4    5    3   25]
 [  77    0   15   17   67]
 [  38    0    8   61  279]
 [  46    1    1   28 3483]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [86]:
linear_model = LinearRegression()


linear_model.fit(X_train_tfidf, Y_train)


Y_pred_linear = linear_model.predict(X_test_tfidf)


mse_linear = mean_squared_error(Y_test, Y_pred_linear)
r2_linear = r2_score(Y_test, Y_pred_linear)

# Print metrics
print("Mean Squared Error (MSE) for Linear Regression:", mse_linear)
print("R-squared score for Linear Regression:", r2_linear)

Mean Squared Error (MSE) for Linear Regression: 3.998442969803879
R-squared score for Linear Regression: -0.43785900791336996


In [87]:
# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, Y_train)
Y_pred_nb = nb_model.predict(X_test_tfidf)

# Metrics
accuracy_nb = accuracy_score(Y_test, Y_pred_nb)
precision_nb = precision_score(Y_test, Y_pred_nb, average='weighted')
recall_nb = recall_score(Y_test, Y_pred_nb, average='weighted')
f1_nb = f1_score(Y_test, Y_pred_nb, average='weighted')
conf_matrix_nb = confusion_matrix(Y_test, Y_pred_nb)

print("NB Performance:")
print("Accuracy:", accuracy_nb)
print("Precision:", precision_nb)
print("Recall:", recall_nb)
print("F1 Score:", f1_nb)
print("Confusion Matrix:\n", conf_matrix_nb)

NB Performance:
Accuracy: 0.8292061179898034
Precision: 0.7225495353197103
Recall: 0.8292061179898034
F1 Score: 0.7708490133443797
Confusion Matrix:
 [[1019    0    0    0  204]
 [  89    0    0    0   59]
 [  57    0    0    0  119]
 [  20    0    0    0  366]
 [  24    0    0    0 3535]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_tfidf, Y_train)
Y_pred_dt = dt_model.predict(X_test_tfidf)

# Metrics
accuracy_dt = accuracy_score(Y_test, Y_pred_dt)
precision_dt = precision_score(Y_test, Y_pred_dt, average='weighted')
recall_dt = recall_score(Y_test, Y_pred_dt, average='weighted')
f1_dt = f1_score(Y_test, Y_pred_dt, average='weighted')
conf_matrix_dt = confusion_matrix(Y_test, Y_pred_dt)

print("Decision Tree Performance:")
print("Accuracy:", accuracy_dt)
print("Precision:", precision_dt)
print("Recall:", recall_dt)
print("F1 Score:", f1_dt)
print("Confusion Matrix:\n", conf_matrix_dt)

Decision Tree Performance:
Accuracy: 0.8486890021849963
Precision: 0.8439238348450196
Recall: 0.8486890021849963
F1 Score: 0.8457305716611339
Confusion Matrix:
 [[1021   39   24   25  114]
 [  34   74   11    6   23]
 [  41    4   86    7   38]
 [  32    7   14  183  150]
 [ 121   19   37   85 3297]]


In [89]:
# Support Vector Machine
svm_model = SVC()
svm_model.fit(X_train_tfidf, Y_train)
Y_pred_svm = svm_model.predict(X_test_tfidf)

# Metrics
accuracy_svm = accuracy_score(Y_test, Y_pred_svm)
precision_svm = precision_score(Y_test, Y_pred_svm, average='weighted')
recall_svm = recall_score(Y_test, Y_pred_svm, average='weighted')
f1_svm = f1_score(Y_test, Y_pred_svm, average='weighted')
conf_matrix_svm = confusion_matrix(Y_test, Y_pred_svm)

print("SVM Performance:")
print("Accuracy:", accuracy_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1 Score:", f1_svm)
print("Confusion Matrix:\n", conf_matrix_svm)

SVM Performance:
Accuracy: 0.8816460305899491
Precision: 0.8840261203225416
Recall: 0.8816460305899491
F1 Score: 0.8540546006974673
Confusion Matrix:
 [[1167    2    1    0   53]
 [ 107   21    1    0   19]
 [  71    0   50    3   52]
 [  28    0    2   89  267]
 [  35    0    1    8 3515]]
